In [3]:
import spacy
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from  sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from spacy.lang.en.stop_words import STOP_WORDS

In [4]:
len(STOP_WORDS)

326

In [6]:
nlp = spacy.load("en_core_web_sm")
doc = nlp("We just opened our wings, the flying part is coming soon")
for token in doc:
  if token.is_stop:
    print(token)

We
just
our
the
part
is


In [23]:
# now create list of non_stopwords
def preprocessing(text):
  doc = nlp(text)
  non_stopwords = [token.text for token in doc if not token.is_stop and not token.is_punct]
  return ' '.join(non_stopwords)


In [14]:
import pandas as pd
df = pd.read_json('doj_press.json',lines= True,encoding='utf-8-sig')
df

,id,title,contents,date,topics,components
0,None,Convicted Bomb Plotter Sentenced to 30 Years,"PORTLAND, Oregon. – Mohamed Osman Mohamud, 23,...",2014-10-01T00:00:00-04:00,[],[National Security Division (NSD)]
1,12-919,$1 Million in Restitution Payments Announced t...,WASHINGTON – North Carolina’s Waccamaw River...,2012-07-25T00:00:00-04:00,[],[Environment and Natural Resources Division]
2,11-1002,$1 Million Settlement Reached for Natural Reso...,BOSTON– A $1-million settlement has been...,2011-08-03T00:00:00-04:00,[],[Environment and Natural Resources Division]
3,10-015,10 Las Vegas Men Indicted \r\nfor Falsifying V...,WASHINGTON—A federal grand jury in Las Vegas...,2010-01-08T00:00:00-05:00,[],[Environment and Natural Resources Division]
4,18-898,$100 Million Settlement Will Speed Cleanup Wor...,"The U.S. Department of Justice, the U.S. Envir...",2018-07-09T00:00:00-04:00,[Environment],[Environment and Natural Resources Division]
...,...,...,...,...,...,...
13082,16-735,Yuengling to Upgrade Environmental Measures to...,The Department of Justice and the U.S. Environ...,2016-06-23T00:00:00-04:00,[Environment],[Environment and Natural Resources Division]
13083,10-473,Zarein Ahmedzay Pleads Guilty to Terror Violat...,The Justice Department announced that Zarein...,2010-04-23T00:00:00-04:00,[],[Office of the Attorney General]
13084,17-045,Zimmer Biomet Holdings Inc. Agrees to Pay $17....,Subsidiary Agrees to Plead Guilty to Violating...,2017-01-12T00:00:00-05:00,[Foreign Corruption],"[Criminal Division, Criminal - Criminal Fraud ..."
13085,17-252,ZTE Corporation Agrees to Plead Guilty and Pay...,ZTE Corporation has agreed to enter a guilty p...,2017-03-07T00:00:00-05:00,"[Asset Forfeiture, Counterintelligence and Exp...","[National Security Division (NSD), USAO - Texa..."


In [17]:
df = df[df['topics'].str.len() != 0]

In [21]:
df = df.head(100)

In [24]:
df['contents_new'] = df['contents'].apply(preprocessing)
df.head()

,id,title,contents,date,topics,components,contents_new
4,18-898,$100 Million Settlement Will Speed Cleanup Wor...,"The U.S. Department of Justice, the U.S. Envir...",2018-07-09T00:00:00-04:00,[Environment],[Environment and Natural Resources Division],U.S. Department Justice U.S. Environmental Pro...
7,14-1412,14 Indicted in Connection with New England Com...,A 131-count criminal indictment was unsealed t...,2014-12-17T00:00:00-05:00,[Consumer Protection],[Civil Division],131 count criminal indictment unsealed today B...
19,17-1419,2017 Southeast Regional Animal Cruelty Prosecu...,The United States Attorney’s Office for the Mi...,2017-12-14T00:00:00-05:00,[Environment],"[Environment and Natural Resources Division, U...",United States Attorney Office Middle District ...
22,15-1562,21st Century Oncology to Pay $19.75 Million to...,"21st Century Oncology LLC, has agreed to pay $...",2015-12-18T00:00:00-05:00,"[False Claims Act, Health Care Fraud]",[Civil Division],21st Century Oncology LLC agreed pay $ 19.75 m...
23,17-1404,21st Century Oncology to Pay $26 Million to Se...,21st Century Oncology Inc. and certain of its ...,2017-12-12T00:00:00-05:00,"[Health Care Fraud, False Claims Act]","[Civil Division, USAO - Florida, Middle]",21st Century Oncology Inc. certain subsidiarie...


In [25]:
text = '''
Thor: Love and Thunder is a 2022 American superhero film based on Marvel Comics featuring the character Thor, produced by Marvel Studios and
distributed by Walt Disney Studios Motion Pictures. It is the sequel to Thor: Ragnarok (2017) and the 29th film in the Marvel Cinematic Universe (MCU).
The film is directed by Taika Waititi, who co-wrote the script with Jennifer Kaytin Robinson, and stars Chris Hemsworth as Thor alongside Christian Bale, Tessa Thompson,
Jaimie Alexander, Waititi, Russell Crowe, and Natalie Portman. In the film, Thor attempts to find inner peace, but must return to action and recruit Valkyrie (Thompson),
Korg (Waititi), and Jane Foster (Portman)—who is now the Mighty Thor—to stop Gorr the God Butcher (Bale) from eliminating all gods.
'''

# **Exercise**

In [38]:
nlp  = spacy.load('en_core_web_sm')
doc = nlp(text)
tokens = [token for token in doc]
stop_words = [token for token in doc if token.is_stop]
total_count = len(tokens)
total_stopword_count = len(stop_words)
print(f'Total words = {total_count} | Total stop words = {total_stopword_count}')
print(f'Percentage of stopwords in text = {(total_stopword_count / total_count) * 100} %')

Total words = 160 | Total stop words = 40
Percentage of stopwords in text = 25.0 %


In [42]:
def preprocess(text):
    doc = nlp(text)
    no_stop_words = [token.text for token in doc if not token.is_stop]
    return " ".join(no_stop_words)

nlp.vocab['not'].is_stop = False
positive_text = preprocess('this is a good movie')
negative_text = preprocess('this is not a good movie')


#step3: finally print those 2 transformed texts
print(f"Text1: {positive_text}")
print(f"Text2: {negative_text}")

Text1: good movie
Text2: not good movie


In [56]:
text2 = ''' The India men's national cricket team, also known as Team India or the Men in Blue, represents India in men's international cricket.
It is governed by the Board of Control for Cricket in India (BCCI), and is a Full Member of the International Cricket Council (ICC) with Test,
One Day International (ODI) and Twenty20 International (T20I) status. Cricket was introduced to India by British sailors in the 18th century, and the
first cricket club was established in 1792. India's national cricket team played its first Test match on 25 June 1932 at Lord's, becoming the sixth team to be
granted test cricket status.
'''
doc = nlp(text2)

not_stopwords = [token.text for token in doc if not token.is_stop or not token.is_punct]
ans_dict = {}
for word in not_stopwords:
  if word != '\n' and word != ' ':
    if word not in ans_dict:
      ans_dict[word] = 1
    else:
      ans_dict[word] += 1
max_count = 0
for k,v in ans_dict.items():
  max_count = max(max_count , v)
max_freq_word = max(ans_dict.keys(), key=(lambda key: ans_dict[key]))


max_freq_word

'India'